In [1]:
import json
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.axes import Axes
from matplotlib.container import BarContainer
import numpy as np
from pathlib import Path

In [2]:
from deepjanus_mnist.mnist_individual import MNISTIndividual
from deepjanus_mnist.mnist_member import MNISTMember
from deepjanus_mnist.mnist_problem import MNISTProblem
from deepjanus_mnist.mnist_config import MNISTConfig
from deepjanus_mnist.image_tools import bitmap_to_svg

data = {}

data_path = Path('./MNIST')
experiment_types = data_path.glob('*/')
for experiment_type in experiment_types:
    if not experiment_type.is_dir():
        continue

    type_data = {}
    
    experiments = experiment_type.glob('*/')
    for experiment in experiments:
        if not experiment.is_dir():
            continue
        
        experiment_data = {}
        
        with open(experiment.joinpath('report.json'), 'r') as f:
            report = json.load(f)
            experiment_data['time'] = report['time']
        
        archive = []
        solutions = experiment.glob('archive/*.json')
        for solution_path in solutions:
            with open(solution_path, 'r') as f:
                solution = json.load(f)
            archive.append(solution)
        experiment_data['archive'] = archive
        
        type_data[str(experiment.name)] = experiment_data
    
    data[str(experiment_type.name)] = type_data
    print(f'Loaded {experiment_type.name} data')

Loaded 20_unsafe data
Loaded 50_unsafe data
Loaded 60_unsafe data
Loaded 80_unsafe data
